In [ ]:
# importing libraries
import pandas as pd

In [ ]:
# default variables
tech_layoffs = pd.read_csv('../data/tech_layoffs.csv', sep=',')

In [9]:
# showing random rows to pre-analyze the data
tech_layoffs.sample(5)

,company,total_layoffs,impacted_workforce_percentage,reported_date,industry,headquarter_location,sources,status,additional_notes
257,Subspace,Unclear,100,5/13/2022,Media/entertainment,Los Angeles,LinkedIn,Private,NaN
10,New Relic,110,5,8/18/2022,"productivity, enterprise software","Bay Area, CA",MarketWatch,Private,NaN
379,Zillow,300,5,10/26/2022,PropTech,Seattle,TechCrunch,Public,NaN
298,Tempo Automation,54,Unclear,8/17/2022,automotive,San Francisco,San Francisco Business Times,Private,NaN
223,Ocavu,20,48,11/10/2022,Cryptocurrency,"Lehi, UT",Salt Lake Tribune,Private,NaN


# **First Basic Analysis**
### **Objectives:**
- Discover how many unique companies are listed on this dataset
- Check which companies had more and fewer layoff
- 

In [14]:
tech_layoffs[['company']].nunique()

company     477
industry    289
dtype: int64

# Analysis 1) Rank which industry had most layoff 

# Analysis 2)  